In [ ]:
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets
import langchain
from langchain.callbacks import StdOutCallbackHandler
from langchain.text_splitter import NLTKTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import traceback
from retrying import retry
import requests
import json
import base64
import tiktoken

def print_red(text):
    print(f"\033[91m {text}\033[00m")

# Convert JSON string to Python Dictionary
def transform_json_string(json_string):
  json_response = json.loads(json_string)
  return json_response

# flatten list into string
def flatten_list(list):
  flat_string = ""
  for i in list:
    flat_string += "\n" + i
  return flat_string

# print in markdown
def print_md(string):
    display(Markdown(string))

# print json formatted dictionary
def print_json(dictionary):
    print(json.dumps(dictionary, indent=4))


def count_tokens(count_this_string):
  enc = tiktoken.encoding_for_model("gpt-4")
  tokens = enc.encode(count_this_string)
  tokenCount = len(tokens)
  return tokenCount


policy_a_tron = """\
    ____        __             _____ __
   / __ \____  / (_)______  __/ ___// /_____  _________ ___
  / /_/ / __ \/ / / ___/ / / /\__ \/ __/ __ \/ ___/ __ `__ \\
 / ____/ /_/ / / / /__/ /_/ /___/ / /_/ /_/ / /  / / / / / /
/_/    \____/_/_/\___/\__, //____/\__/\____/_/  /_/ /_/ /_/
PolicyStorm v2       /____/
"""

def system_message_template(expert, response_format):
    position = expert['position']
    organisation = expert['organisation']
    expertise = expert['expertise']
    papers = expert['papers']
    name = expert['name']
    system_message = (f"""\
You are {name}, a highly experienced {position} with {organisation}.
You have authored a variety of high quality {papers}.
Your academic writing leverages your highly regarded expertise in {expertise}.
You provide clear, concise narratives, maintaining a balanced and objective \
approach to assessing policy problems and arriving at solutions step by step.

Never use backticks (`) or similar delimiters in your output.
{response_format}""")
    return system_message

markdown_response = "Output in markdown only"

def json_response_template():
    json_response = (f"""

Only output JSON formatted text. Never chat with user.
The first character of the output must be a curly brace or a square bracket (as per the example).
The last character of the output must be a curly brace or a square bracket (as per the example).
Follow the structure of any example closely. Do not add, remove, or change keys.
""")
    return json_response



total_script_cost = 0

@retry(stop_max_attempt_number=10, wait_fixed=2000)  # adjust parameters as needed
def call_llm(expert, prompt, temp):
    api_key = password_widget.value

    try:
        system_template = "{expert}"
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        human_template = "{prompt}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
        handler = StdOutCallbackHandler()
        chat = ChatOpenAI(model='gpt-4', verbose=True, temperature=temp, openai_api_key=api_key, streaming=False, callbacks=[StreamingStdOutCallbackHandler()])
        #chain = LLMChain(llm=chat, prompt=chat_prompt, verbose=True, callbacks=[handler]) # This prints prompts.
        chain = LLMChain(llm=chat, prompt=chat_prompt) # This doesn't print prompts.
        completion = chain.run(expert=expert, prompt=prompt)

        count_total_tokens(expert, prompt, completion)

        data = {
            'system_message': expert,
            'human_message': prompt,
            'completion': completion
        }

        return completion

    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()  # this will print the full traceback
        raise   # this will re-raise the error, triggering our retry logic

def count_total_tokens(system_message, human_message, completion):
    global total_script_cost
    system_message_tokens = count_tokens(system_message)
    human_message_tokens = count_tokens(human_message)
    completion_tokens = count_tokens(completion)

    input_tokens = system_message_tokens + human_message_tokens
    output_tokens = completion_tokens

    cost_of_input = (input_tokens / 1000) * 0.03
    cost_of_output = (completion_tokens / 1000) * 0.06
    total_cost = cost_of_input + cost_of_output
    #print_red(f"Cost: ${total_cost:.2f}")
    total_script_cost += total_cost  # add the total_cost to the total_script_cost


style = {'description_width': 'initial'}

text_area = widgets.Textarea(
    value='',
    placeholder= policy_a_tron,
    layout=widgets.Layout(width="490px", height="125px"),
    style=style,
)

button = widgets.Button(
    description="Submit Your Policy Problem",
    button_style='info',
    tooltip='Click to Get Experts',
    icon='id-badge',
    layout=widgets.Layout(width="490px", height="40px")
)

out=widgets.Output()

def on_text_change(change):
    global Problem
    Problem = change['new']

text_area.observe(on_text_change, names='value')
debate_history = ""
debate_summary = ""
def commence_debate(section_for_debate, llm_temp, debate_summary, debate_history):
  for i in range (2):
    debate_history = assign_debate_task('Default Expert', debate_brainstorm_template('Default Expert', questioner_role, section_for_debate, debate_history, debate_summary), 0.4, markdown_response)
    debate_history += "\n\n" + assign_debate_task('Expert 1', debate_brainstorm_template('Expert 1', answering_role, section_for_debate, debate_history, debate_summary), 0.2, markdown_response)
    debate_history += "\n\n" + assign_debate_task('Expert 2', debate_brainstorm_template('Expert 2', answering_role, section_for_debate, debate_history, debate_summary), 0.2, markdown_response)
    debate_history += "\n\n" + assign_debate_task('Expert 3', debate_brainstorm_template('Expert 3', answering_role, section_for_debate, debate_history, debate_summary), 0.2, markdown_response)
    debate_history += "\n\n" + assign_debate_task('Expert 4', debate_brainstorm_template('Expert 4', answering_role, section_for_debate, debate_history, debate_summary), 0.8, markdown_response)
    debate_summary += assign_debate_task('Default Expert', debate_brainstorm_template('Default Expert', summarizor_role, section_for_debate, debate_history, debate_summary), 0.1, markdown_response)
    debate_history= ""
  return debate_summary

def replace_authors_and_start_debate(b):
  with out:
    display(Markdown("## Expert Selection"))  # use Markdown function
    print("Please wait. Reviewing CVs to select your experts...")
    replace_all_authors(text_area.value)
    display(Markdown("## Brainstorming Session"))  # use Markdown function
    print("Please wait. Commencing brainstorming session")
    commence_debate(text_area.value, 0.2, debate_history, debate_summary)
    print_red(f"Total Cost of Brainstorm: ${total_script_cost:.2f}")

button.on_click(replace_authors_and_start_debate)

password_widget = widgets.Password(
    value='',
    placeholder='API Key',
    description='',
    disabled=False,
    layout=widgets.Layout(width="490px"),
    style=style,
)

center_layout = widgets.Layout(display='flex',
                               flex_flow='column',
                               align_items='center',
                               width='100%')

widgets_in_a_column = widgets.VBox([text_area, password_widget, button], layout=center_layout)

display(widgets_in_a_column)

def replace_author(author_for_system_message, response_format, topic, author_key_to_replace, llm_temp):
    system_message = system_message_template(author_for_system_message, response_format)
    human_message = (f"create a new profile for the ideal expert to help us deal with the following policy problem: \n '{topic}'. \n Follow the Example for your output. Ensure your output contains keys for 'name', 'position', 'organisation', 'expertise', and 'papers'. \n {format_example_expert}")
    completion = call_llm(system_message, human_message, llm_temp)
    json_completion = transform_json_string(completion)
    experts_dict[author_key_to_replace] = json_completion
    with out:
        for title, detail in json_completion.items():
            display(Markdown(f"**{title.title()}:** {detail}"))  # use Markdown function
        display(Markdown("<br>"))  # use Markdown function


def replace_all_authors(topic):
    author_for_system_message = experts_dict['Default Expert']
    response_format = json_response_template()
    replace_author(author_for_system_message, response_format, (f"{topic}. The ideal expert will have significant experience working at the regulatory agency most relevant to the policy problem"), 'Expert 1', 0.2)
    replace_author(author_for_system_message, response_format, (f"{topic}. The ideal expert will have significant legal experience relating to the policy problem and work for a white shoe law firm"), 'Expert 2', 0.2)
    replace_author(author_for_system_message, response_format, (f"{topic}. The ideal expert will be a key industry stakeholder with significant experience in a relevant technical or technology field"), 'Expert 3', 0.2)
    replace_author(author_for_system_message, response_format, (f"{topic}. The ideal expert will be an industry lobbyist or industry association with a strong technical or technology background"), 'Expert 4', 0.2)



def debate_brainstorm_template(expert, debate_role, section_for_debate, debate_history, debate_summary):
    debate_brainstorm_prompt = (f"""\
You are engaged in a rigorous academic brainstorm session on a topic related \
to the following policy problem:
- {section_for_debate}

Your overall goal (and the goal of the other members of the team) is to ensure \
a high quality brainstorm session related to the policy problem. Take it in turns to comment \
on each other's views.

{debate_role}

## Summary of Earlier Sessions:
```
{debate_summary}
```

## Guidelines for Output:
- **Output Style:** Academic, assertive, concise but comprehensive. Use no more words than necessary.
- **Argument Foundation:** Grounded in valid and relevant justifications
- **Validity:** If your position is not sustainable, concede the argument.

Ensure your messages to the other members of the team are prefaced with your name. For example:\
'''**{experts_dict[expert]['name']}**:
[message]'''
---
Participants are reminded to be concise. Use no more words than necessary to make your argument.
## This Session:
{debate_history}



""")
    return debate_brainstorm_prompt

questioner_role = """
Your specific role in the session is to examine the relevant subject matter by \
using the Socratic method of questioning. Your goal is to stimulate critical \
thinking, elicit truth, and expose contradictions. You do this by asking questions \
for others in the group to answer."""

consensus_role = """
Your specific role in the session is to help the group toward consensus. \
You should do so by clearly identifying the issues that are agreed and the \
issues that are in dispute. Focus the group on the areas of disagreement.
"""

answering_role = 'Your specific role in the session is to answer the questions \
posed by other members of the group. You should note where you disagree with \
one of the other team members. You should justify your diagreement. Robust \
debate is expected but consensus must ultimately be reached. Cencede your weakest \
points when presented with compelling justification. After answering \
you should reflect on your views about the policy problem. Have they changed?'


summarizor_role = '''Your specific role in the session is to summarize the \
brainstorm session. Your summary should be concise, preserving the key points \
and overall meaning of the session. You must clearly identify who disagrees on \
what topic and why. Your summary should include a list of all topics where \
the members have reaches consensus. Your final paragraph must be a conclusion \
that identifies the areas in dispute that will be the focus of the next session.
Start your response with '### Summary of Session {i}'''


format_example_expert = """\
{
    'name': 'Steven Yellen',
    'position': 'Senior Policy Advisor & Financial Regulations Expert',
    'organisation': 'Department of Treasury',
    'expertise': 'political science, law, public policy, and economics, specializing in the formulation, analysis, and implementation of financial service policies',
    'papers': 'policy statements, consultation papers, and journal articles on a range of financial services policy issues, including regulatory compliance, financial service innovations, the impact of policies on financial services and financial markets'
}"""

experts_dict = {
    'Default Expert':
    {
        'name': 'Dr. Chris Adamek',
        'position': 'Senior Policy Advisor & Financial Regulations Expert',
        'organisation': 'Department of Treasury',
        'expertise': 'political science, law, public policy, and economics, specializing in the formulation, analysis, and implementation of financial service policies',
        'papers': 'policy statements, consultation papers, and journal articles on a range of financial services policy issues, including regulatory compliance, financial service innovations, the impact of policies on financial services and financial markets'
    }
}

def assign_debate_task(expert, debate_template, temp, response_type):
    system_message = system_message_template(experts_dict[expert], markdown_response)
    human_message = debate_template
    task_completion = call_llm(system_message, human_message, temp)
    with out:
        display(Markdown("\n"))  # use Markdown function
        display(Markdown(task_completion))  # use Markdown function

    return task_completion



In [ ]:
display(out)